# Compare single-loop and double-loop downsampling simulation speeds

#### Chris Porras

In [64]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
rc = {'lines.linewidth': 3, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5',
      'axes.font': 'arial'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)
sns.set_palette('colorblind')
%matplotlib inline
from scipy.ndimage.filters import laplace

In [265]:
## Initialize dict of all kwargs
## to be updated by just redefining vars
## i.e. change mu by setting mu = 10**-5
## not by editing dict directly

kwargs={'mu':10**-6,
      's':10**-2,
      'm':10**-1,
      'pop_size':10**4,
      'dims':(2,2),
      'num_intervals':100,
      'n': 2,
      'sigma':1}


Store parameters in a single dictionary inst

In [262]:
def update_kwargs():
    for key in kwargs.keys():
        if key in globals() and globals()[key] != kwargs[key]:
            kwargs[key] = globals()[key]

In [263]:
def calculate_f(f,**kwargs):
    #Wright-Fisher diffusion w/Stepping Stone migration
    df = mu*(1-2*f)-s*f*(1-f) \
    +m*laplace(f,mode='wrap')
    #bounds allele frequencies
    p = np.clip(a= f + df ,a_min=0,a_max=1)
    #genetic drift sampling
    f= np.random.binomial(pop_size,p)/pop_size
    return f

In [264]:
def downsample_f(**kwargs):
    update_kwargs()
    output = np.zeros(tuple([num_intervals])+dims)
    interval = int(1/s)
    num_gens = interval * num_intervals
    f = np.ones(dims)/pop_size
    for i in range(num_gens):
        new_f = calculate_f(f,**kwargs)
        f = new_f
        if (i + 1) % interval == 0:
            output[i//interval] = f
    return output

#### Optimizing downsampling in single-loop

In [850]:
%%timeit

## Old double loop method:
np.random.seed(2)

# Population parameters
pop_size = 10**2
mu = 10**-1
m = 10**-1
num_reps = 1
num_gen = 10**2
dims = (50,1)
s=10**-2

num_intervals = 1000

y = np.array((np.repeat(np.arange(num_intervals-1),(int(1/s)-1)),
             np.ravel([np.arange(int(1/s)-1)]*(num_intervals-1)))).T

q = np.zeros((num_intervals,num_reps)+dims)
q[0] = 1/pop_size
f = np.zeros((int(1/s),num_reps)+dims)

for i,j in y:
    if j == 0:
        f[j] = q[i]

    #Wright-Fisher diffusion w/Stepping Stone migration
    df = mu*(1-2*f[j])-s*f[j]*(1-f[j]) \
    +m*laplace(f[j],mode='wrap')
    #bounds allele frequencies
    p = np.clip(a= f[j] + df ,a_min=0,a_max=1)
    #genetic drift sampling
    f[j+1]= np.random.binomial(pop_size,p)/pop_size
    
    if j+1 == int(1/s)-1:
        q[i+1] = f[-1]

7.41 s ± 83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [267]:
%%timeit

np.random.seed(2)

# Population parameters
pop_size = 10**2
mu = 10**-1
m = 10**-1
num_reps = 1
num_gen = 10**2
dims = (50,1)
s=10**-2

num_intervals = 1000

output = downsample_f(**kwargs)

5.36 ms ± 78.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Single loop much faster!